In [26]:
from shapely.geometry import MultiPoint, LineString, MultiLineString, Point
from shapely.ops import nearest_points, linemerge
import numpy as np
# from libpysal.cg.voronoi import voronoi, voronoi_frames
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.ops import unary_union, polygonize, cascaded_union, split, polygonize_full
from shapely import affinity
from datetime import datetime
from os.path import exists
import pygeos as pg
import os

In [27]:
df_qv = gpd.read_file('gis/SIRGAS_GPKG_quadraviariaed_2017.gpkg')
df_l = gpd.read_file(f'zip://gis/SIRGAS_SHP_logradouronbl.zip!SIRGAS_SHP_logradouronbl/SIRGAS_SHP_logradouronbl.shp')

In [28]:
df_dist = gpd.read_file(f'zip://gis/SIRGAS_SHP_distrito.zip!SIRGAS_SHP_distrito/SIRGAS_SHP_distrito_polygon.shp')
df_represas = gpd.read_file(f'zip://gis/SIRGAS_SHP_represanivelmax.zip!SIRGAS_SHP_represanivelmax').set_crs(epsg=31983)
df_massa_dagua = gpd.read_file(f'zip://gis/SIRGAS_SHP_hidrogramassadagua.zip!SIRGAS_SHP_hidromassadagua/SIRGAS_SHP_hidromassadagua_polygon.shp').set_crs(epsg=31983)

In [29]:
df_cruzamentos = gpd.read_file(f'resultado/pre-processamentos/00_cruzamentos_cidade_sao_paulo.gpkg')

In [221]:
for id, dist in df_dist.iterrows():

    print(f'Processando meio-fio de {dist.ds_nome}')
    if exists(f'resultado/pre-processamentos/{dist.ds_codigo}_meio-fio_{dist.ds_nome}.gpkg'):
        continue

    qv = df_qv.loc[df_qv.qe_tipo == 'Quadra'].clip(dist.geometry)
    qv = gpd.GeoDataFrame(geometry=qv.buffer(0.001))
    l = df_l.clip(dist.geometry.buffer(20))
    # l = gpd.GeoDataFrame(geometry=l)

    cruzamentos = df_cruzamentos.clip(dist.geometry)
    cruzamentos_buffer = gpd.GeoDataFrame(geometry=cruzamentos.buffer(35)).set_crs(epsg=31983)
    
    esquinas = qv.overlay(cruzamentos_buffer, how='intersection').explode(index_parts=False).reset_index()
    # esquinas = gpd.GeoDataFrame(geometry=[esquinas]).explode(index_parts=True).reset_index()

    cruzamento_esquina = gpd.sjoin(esquinas, cruzamentos_buffer, how='left', predicate='intersects').reset_index()
    # cruzamento_esquina = cruzamento_esquina[~cruzamento_esquina.index_right.isna()]

    linhas_corte = MultiLineString([LineString(nearest_points(r.geometry, cruzamentos.loc[r.index_right].geometry)) 
                            for i, r in cruzamento_esquina.iterrows()])

    round = np.vectorize(lambda geom: pg.apply(geom, lambda g: g.round(3)))

    linhas_corte = gpd.GeoDataFrame(geometry=round(gpd.GeoDataFrame(geometry=[linhas_corte]).geometry.values.data)).geometry
    qv_boundary = qv.boundary.unary_union

    linhas_unidas = qv_boundary.union(linhas_corte.unary_union)
    geometrias = [i for i in linemerge(linhas_unidas)]

    vias_union = dist.geometry.difference(qv.buffer(0.30).unary_union)
    qv_union = gpd.GeoDataFrame(geometry=qv.buffer(-0.30)).unary_union

    meios_fios = gpd.GeoDataFrame(geometry=geometrias, crs=31983)

    meio_fio_ponto_medio = gpd.GeoDataFrame(meios_fios.copy(), geometry=[l.interpolate(0.5, normalized=True) for l in meios_fios.geometry]).sjoin_nearest(l, how='left').drop_duplicates()

    meios_fios.loc[meio_fio_ponto_medio['lg_codlog'].index, 'lg_codlog'] = meio_fio_ponto_medio['lg_codlog']
    meios_fios.loc[meio_fio_ponto_medio['lg_seg_id'].index, 'lg_seg_id'] = meio_fio_ponto_medio['lg_seg_id']
    meios_fios.loc[meio_fio_ponto_medio.index, 'lado_par'] = [r.geometry.intersects(l.loc[r.index_right].geometry.buffer(35, single_sided=True)) for i, r in meio_fio_ponto_medio.iterrows()]

    meios_fios = meios_fios.dissolve(by=['lg_codlog', 'lg_seg_id', 'lado_par']).reset_index()

    meios_fios = meios_fios.explode(index_parts=False).reset_index()
    meios_fios = meios_fios[~meios_fios.intersects(vias_union)]
    meios_fios = meios_fios[~meios_fios.intersects(qv_union)]

    meios_fios = meios_fios.dissolve(by=['lg_codlog', 'lg_seg_id', 'lado_par']).reset_index()

    meios_fios.to_file(f'resultado/pre-processamentos/{dist.ds_codigo}_meio-fio_{dist.ds_nome}.gpkg', 
                        driver='GPKG', crs=31983)

    break

Processando meio-fio de MOEMA
Processando meio-fio de PARQUE DO CARMO
